In [47]:
import pandas as pd
import ast
from datetime import timedelta
from haversine import haversine, Unit
from scipy.ndimage import gaussian_filter1d
import numpy as np
import ast
import json
from shapely.geometry import Point, LineString



# read and sort data
tpl = pd.read_csv('./data/finished_tpl.csv')
sp = pd.read_csv('./data/finished_sp.csv')

tpl.sort_values('started', inplace=True)
sp.sort_values('started', inplace=True)
sp['duration'] = (sp.finished-sp.started)/60


# create a linear df with every recorded coord and ts as one line

for i, obj in tpl.iterrows():
    path = ast.literal_eval(obj['path'])
    tpl.at[i,'start_lon'] = path[0][0]
    tpl.at[i,'start_lat'] = path[0][1]
    tpl.at[i,'end_lon'] = path[-1][0]
    tpl.at[i,'end_lat'] = path[-1][1]
tpl['type'] = 'tpl'

sp['start_lon'] = sp['lon']
sp['start_lat'] = sp['lat']
sp['end_lon'] = sp['lon']
sp['end_lat'] = sp['lat']
sp['type'] = 'sp'

cols = ['started_at', 'finished_at', 'start_lon','start_lat','end_lon','end_lat','type']
df = sp[cols].append(tpl[cols])

df['started_at'] = pd.to_datetime(df['started_at'])
df['finished_at'] = pd.to_datetime(df['finished_at'])
df.sort_values('started_at', inplace=True)
df





,started_at,finished_at,start_lon,start_lat,end_lon,end_lat,type
376,2023-01-31 13:36:48.750000+00:00,2023-01-31 23:42:33.538000+00:00,8.555457,47.461529,72.873423,19.097524,tpl
368,2023-01-31 23:42:33.538000+00:00,2023-02-01 00:54:30.468000+00:00,72.874645,19.099977,72.874645,19.099977,sp
369,2023-02-01 01:34:23.001000+00:00,2023-02-01 11:44:32.998000+00:00,72.825227,18.994456,72.825227,18.994456,sp
370,2023-02-01 11:44:32.998000+00:00,2023-02-01 12:50:27.022000+00:00,72.823200,18.998380,72.823200,18.998380,sp
371,2023-02-01 13:06:00.730000+00:00,2023-02-01 13:50:16.321000+00:00,72.825072,18.993581,72.825072,18.993581,sp
...,...,...,...,...,...,...,...
753,2023-07-07 04:44:42.406000+00:00,2023-07-07 04:56:00.499000+00:00,127.803842,34.594460,127.803610,34.592116,tpl
720,2023-07-07 04:56:00.499000+00:00,2023-07-07 05:04:51.854000+00:00,127.803774,34.591762,127.803774,34.591762,sp
754,2023-07-07 05:04:51.854000+00:00,2023-07-07 05:29:59.973000+00:00,127.802455,34.593707,127.803484,34.594783,tpl
755,2023-07-07 05:29:59.973000+00:00,2023-07-07 06:25:38.469000+00:00,127.803478,34.594771,127.741499,34.744213,tpl


In [51]:
df['temp_gap_h'] = (df.started_at.shift(-1)-df.finished_at)/timedelta(hours=2)
df

,started_at,finished_at,start_lon,start_lat,end_lon,end_lat,type,temp_gap_h
376,2023-01-31 13:36:48.750000+00:00,2023-01-31 23:42:33.538000+00:00,8.555457,47.461529,72.873423,19.097524,tpl,0.000000
368,2023-01-31 23:42:33.538000+00:00,2023-02-01 00:54:30.468000+00:00,72.874645,19.099977,72.874645,19.099977,sp,0.332296
369,2023-02-01 01:34:23.001000+00:00,2023-02-01 11:44:32.998000+00:00,72.825227,18.994456,72.825227,18.994456,sp,0.000000
370,2023-02-01 11:44:32.998000+00:00,2023-02-01 12:50:27.022000+00:00,72.823200,18.998380,72.823200,18.998380,sp,0.129682
371,2023-02-01 13:06:00.730000+00:00,2023-02-01 13:50:16.321000+00:00,72.825072,18.993581,72.825072,18.993581,sp,0.000000
...,...,...,...,...,...,...,...,...
753,2023-07-07 04:44:42.406000+00:00,2023-07-07 04:56:00.499000+00:00,127.803842,34.594460,127.803610,34.592116,tpl,0.000000
720,2023-07-07 04:56:00.499000+00:00,2023-07-07 05:04:51.854000+00:00,127.803774,34.591762,127.803774,34.591762,sp,0.000000
754,2023-07-07 05:04:51.854000+00:00,2023-07-07 05:29:59.973000+00:00,127.802455,34.593707,127.803484,34.594783,tpl,0.000000
755,2023-07-07 05:29:59.973000+00:00,2023-07-07 06:25:38.469000+00:00,127.803478,34.594771,127.741499,34.744213,tpl,0.000000


In [52]:
def calculate_speed_fast(df):
    # Convert latitude and longitude to radians
    lat1, lon1 = np.radians(df['end_lat']), np.radians(df['end_lon'])
    lat2, lon2 = np.radians(df['start_lat'].shift(-1)), np.radians(df['start_lon'].shift(-1))

    # Calculate the distance between consecutive rows using the Haversine formula
    R = 6371  # Earth's radius in km
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * \
        np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    df['distance'] = R * c

    return df

df = calculate_speed_fast(df)
df

,started_at,finished_at,start_lon,start_lat,end_lon,end_lat,type,temp_gap_h,distance
376,2023-01-31 13:36:48.750000+00:00,2023-01-31 23:42:33.538000+00:00,8.555457,47.461529,72.873423,19.097524,tpl,0.000000,0.301511
368,2023-01-31 23:42:33.538000+00:00,2023-02-01 00:54:30.468000+00:00,72.874645,19.099977,72.874645,19.099977,sp,0.332296,12.831703
369,2023-02-01 01:34:23.001000+00:00,2023-02-01 11:44:32.998000+00:00,72.825227,18.994456,72.825227,18.994456,sp,0.000000,0.485593
370,2023-02-01 11:44:32.998000+00:00,2023-02-01 12:50:27.022000+00:00,72.823200,18.998380,72.823200,18.998380,sp,0.129682,0.568726
371,2023-02-01 13:06:00.730000+00:00,2023-02-01 13:50:16.321000+00:00,72.825072,18.993581,72.825072,18.993581,sp,0.000000,0.009072
...,...,...,...,...,...,...,...,...,...
753,2023-07-07 04:44:42.406000+00:00,2023-07-07 04:56:00.499000+00:00,127.803842,34.594460,127.803610,34.592116,tpl,0.000000,0.042129
720,2023-07-07 04:56:00.499000+00:00,2023-07-07 05:04:51.854000+00:00,127.803774,34.591762,127.803774,34.591762,sp,0.000000,0.247693
754,2023-07-07 05:04:51.854000+00:00,2023-07-07 05:29:59.973000+00:00,127.802455,34.593707,127.803484,34.594783,tpl,0.000000,0.001406
755,2023-07-07 05:29:59.973000+00:00,2023-07-07 06:25:38.469000+00:00,127.803478,34.594771,127.741499,34.744213,tpl,0.000000,0.164622


In [58]:
df['next_type']= df['type'].shift(-1)
df['next_start']= df['started_at'].shift(-1)

df

,started_at,finished_at,start_lon,start_lat,end_lon,end_lat,type,temp_gap_h,distance,next_type,next_start
376,2023-01-31 13:36:48.750000+00:00,2023-01-31 23:42:33.538000+00:00,8.555457,47.461529,72.873423,19.097524,tpl,0.000000,0.301511,sp,2023-01-31 23:42:33.538000+00:00
368,2023-01-31 23:42:33.538000+00:00,2023-02-01 00:54:30.468000+00:00,72.874645,19.099977,72.874645,19.099977,sp,0.332296,12.831703,sp,2023-02-01 01:34:23.001000+00:00
369,2023-02-01 01:34:23.001000+00:00,2023-02-01 11:44:32.998000+00:00,72.825227,18.994456,72.825227,18.994456,sp,0.000000,0.485593,sp,2023-02-01 11:44:32.998000+00:00
370,2023-02-01 11:44:32.998000+00:00,2023-02-01 12:50:27.022000+00:00,72.823200,18.998380,72.823200,18.998380,sp,0.129682,0.568726,sp,2023-02-01 13:06:00.730000+00:00
371,2023-02-01 13:06:00.730000+00:00,2023-02-01 13:50:16.321000+00:00,72.825072,18.993581,72.825072,18.993581,sp,0.000000,0.009072,tpl,2023-02-01 13:50:16.321000+00:00
...,...,...,...,...,...,...,...,...,...,...,...
753,2023-07-07 04:44:42.406000+00:00,2023-07-07 04:56:00.499000+00:00,127.803842,34.594460,127.803610,34.592116,tpl,0.000000,0.042129,sp,2023-07-07 04:56:00.499000+00:00
720,2023-07-07 04:56:00.499000+00:00,2023-07-07 05:04:51.854000+00:00,127.803774,34.591762,127.803774,34.591762,sp,0.000000,0.247693,tpl,2023-07-07 05:04:51.854000+00:00
754,2023-07-07 05:04:51.854000+00:00,2023-07-07 05:29:59.973000+00:00,127.802455,34.593707,127.803484,34.594783,tpl,0.000000,0.001406,tpl,2023-07-07 05:29:59.973000+00:00
755,2023-07-07 05:29:59.973000+00:00,2023-07-07 06:25:38.469000+00:00,127.803478,34.594771,127.741499,34.744213,tpl,0.000000,0.164622,sp,2023-07-07 06:25:38.469000+00:00


In [54]:
import plotly.express as px

fig = px.box(df, y = 'distance')
fig.show()

In [64]:
from shapely.geometry import Point, LineString

df.loc[df['distance']>0.13]
df['next_st_lon'] = df['start_lon'].shift(-1)
df['next_st_lat'] = df['start_lat'].shift(-1)

df.loc[df['distance']>0.13]
geoms = []
for i, obj in df.iterrows():
    start_pt = Point(obj['end_lon'], obj['end_lat'])
    end_pt = Point(obj['next_st_lon'], obj['next_st_lat'])
    df.at[i, 'types'] = obj['type'] +' '+ obj['next_type']
    try:
        df.at[i, 'gap_geom'] = LineString([start_pt, end_pt])
        df.at[i, 'types'] = obj['type'] +' '+ obj['next_type']

    except:
        df.at[i, 'gap_geom'] = LineString()
        df.at[i, 'types'] = 'nothing'



df

TypeError: can only concatenate str (not "float") to str